In [46]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from dotenv import load_dotenv

import json

load_dotenv()

True

## Init models

In [47]:
sentiment_analysis_model = ChatGroq(model="llama-3.3-70b-versatile", model_kwargs={"response_format": {"type": "json_object"}})
feature_extraction_model = ChatGroq(model="gemma2-9b-it",  model_kwargs={"response_format": {"type": "json_object"}})

## Create prompt templates

In [65]:
sentiment_analysis_system_message = SystemMessage("You are an emotion analysis bot. Your job is to perform sentiment analysis on the given text and identify the top 5 emotions present. " \
                                                    "Your response should be in the form of a JSON object with a single attribute called 'sentiments' " \
                                                    "which contains an array of the top 5 emotions in order from most prevalent to least prevalent." \
                                                    " Each emotion should be an ordered pair of strings with the first element as the emotion and the second" \
                                                    " as a rating of the emotion. The rating can only be 'positive', 'neutral', or 'negative'.")
sentiment_analysis_template = ChatPromptTemplate.from_messages([sentiment_analysis_system_message, ("user", "{input}")])

In [49]:
action_extraction_system_message = SystemMessage("You are a textual analysis bot. Your job is to find every phrase in the given text that may indicate the intent for action. " \
                                                    "Return a JSON object containing every phrase that you find to be actionable. Be liberal with your analysis, as these can be phrases without verbs.")
action_extraction_template = ChatPromptTemplate.from_messages([action_extraction_system_message, ("user", "{input}")])

In [ ]:
danger_extraction_system_message = SystemMessage("You are a text extraction bot. Your job is to find and extract every word in the given text which" \
                                                    " may convey danger, violence, or harm. Only extract these words and return them in the form of a JSON object" \
                                                    " with a single attribute called 'danger_words' containing an array of the words you have extracted ordered from"
                                                    " most to least dangerous/violent/harmful. This list can be empty if no danger words are found.")
danger_extraction_template = ChatPromptTemplate.from_messages([danger_extraction_system_message, ("user", "{input}")])

## Invoke models

In [51]:
sentiment_prompt = sentiment_analysis_template.invoke({"input": "I am so happy today, I feel like I can conquer the world!"})
sentiment_prompt

ChatPromptValue(messages=[SystemMessage(content="You are an emotion analysis bot. Your job is to perform sentiment analysis on the given text and identify the top 5 emotions present. Your response should be in the form of a JSON object with a single attribute called 'sentiments' which contains an array of the top 5 emotions in order from most prevalent to least prevalent. Each emotion should be an ordered pair as a tuple, with the first element as the emotion itself and the second as a rating of the emotion. The rating can only be 'positive', 'neutral', or 'negative'.", additional_kwargs={}, response_metadata={}), HumanMessage(content='I am so happy today, I feel like I can conquer the world!', additional_kwargs={}, response_metadata={})])

In [52]:
action_extraction_prompt = action_extraction_template.invoke({"input": "I am going to the store to buy some milk. I also need to pick up some bread and eggs. Then at 7:30 I will go to the gym for a workout."})
action_extraction_prompt

ChatPromptValue(messages=[SystemMessage(content='You are a textual analysis bot. Your job is to find every phrase in the given text that may indicate the intent for action. Return a JSON object containing every phrase that you find to be actionable. Be liberal with your analysis, as these can be phrases without verbs.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I am going to the store to buy some milk. I also need to pick up some bread and eggs. Then at 7:30 I will go to the gym for a workout.', additional_kwargs={}, response_metadata={})])

In [53]:
danger_extraction_prompt = danger_extraction_template.invoke({"input": "This is going to be fun, time to spar with my friends in the boxing ring. I really think I can beat them!"})
danger_extraction_prompt

ChatPromptValue(messages=[SystemMessage(content="You are a text extraction bot. Your job is to find and extract every word in the given text which may convey danger, violence, or harm. Only extract these words and return them in the form of a JSON object with a single attribute called 'danger_words' containing an array of the words you have extracted ordered from most to least dangerous/violent/harmful. This may be empty if no danger words are found.", additional_kwargs={}, response_metadata={}), HumanMessage(content='This is going to be fun, time to spar with my friends in the boxing ring. I really think I can beat them!', additional_kwargs={}, response_metadata={})])

In [54]:
sentiment_analysis = sentiment_analysis_model.invoke(sentiment_prompt)
print(sentiment_analysis.content)

{
   "sentiments": [
      ["Happiness", "positive"],
      ["Optimism", "positive"],
      ["Excitement", "positive"],
      ["Confidence", "positive"],
      ["Euphoria", "positive"]
   ]
}


In [ ]:
action_extraction = feature_extraction_model.invoke(action_extraction_prompt)
print(action_extraction.content)

{
  "actionable_phrases": [
    "going to the store to buy some milk",
    "need to pick up some bread and eggs",
    "will go to the gym for a workout"
  ]
}


ChatPromptValue(messages=[SystemMessage(content="You are an emotion analysis bot. Your job is to perform sentiment analysis on the given text and identify the top 5 emotions present. Your response should be in the form of a JSON object with a single attribute called 'sentiments' which contains an array of the top 5 emotions in order from most prevalent to least prevalent. Each emotion should be an ordered pair as a tuple, with the first element as the emotion itself and the second as a rating of the emotion. The rating can only be 'positive', 'neutral', or 'negative'.", additional_kwargs={}, response_metadata={}), HumanMessage(content='', additional_kwargs={}, response_metadata={})])

In [78]:
action_phrases = json.loads(action_extraction.content)

actions = ".\n".join(action_phrases["actionable_phrases"])
print(actions)

action_sentiment_prompt = sentiment_analysis_template.invoke({"input": actions})
print(action_sentiment_prompt)
action_sentiment_analysis = sentiment_analysis_model.invoke(action_sentiment_prompt)
print(action_sentiment_analysis.content)

going to the store to buy some milk.
need to pick up some bread and eggs.
will go to the gym for a workout
messages=[SystemMessage(content="You are an emotion analysis bot. Your job is to perform sentiment analysis on the given text and identify the top 5 emotions present. Your response should be in the form of a JSON object with a single attribute called 'sentiments' which contains an array of the top 5 emotions in order from most prevalent to least prevalent. Each emotion should be an ordered pair of strings with the first element as the emotion and the second as a rating of the emotion. The rating can only be 'positive', 'neutral', or 'negative'.", additional_kwargs={}, response_metadata={}), HumanMessage(content='going to the store to buy some milk.\nneed to pick up some bread and eggs.\nwill go to the gym for a workout', additional_kwargs={}, response_metadata={})]
{
   "sentiments": [
      ["Neutrality", "neutral"],
      ["Determination", "positive"],
      ["Routine", "neutral

In [80]:
danger_words_prompt = danger_extraction_template.invoke({"input": actions})
actionable_danger_words = feature_extraction_model.invoke(danger_words_prompt)
print(actionable_danger_words.content)

{
  "danger_words": []
}


In [ ]:
danger_extraction = feature_extraction_model.invoke(danger_extraction_prompt)
print(danger_extraction.content)

{
  "danger_words": [
    "spar",
    "boxing",
    "beat"
  ]
}
